# Quick Start

See the API for more detailed information, examples, formulas, and references for each function.

In [2]:
import numpy as np
import xarray as xr
import xskillscore as xs

Here, we generate some sample gridded data. Our data has three time steps, and a 4x5 latitude/longitude grid. `obs` replicates some verification data and `fct` some forecast (e.g. from a statistical or dynamical model).

In [3]:
obs = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.arange(4),
           np.arange(5),
       ],
       dims=["time", "lat", "lon"],
   )
fct = obs.copy()
fct.values = np.random.rand(3, 4, 5)

## Deterministic Metrics

`xskillscore` offers a suite of correlation-based and distance-based deterministic metrics.

### Correlation-Based 

* Pearson Correlation (`pearson_r`)
* Pearson Correlation p value (`pearson_r_p_value`)
* Pearson Correlation effective p value (`pearson_r_eff_p_value`)
* Spearman Correlation (`spearman_r`)
* Spearman Correlation p value (`spearman_r_p_value`)
* Spearman Correlation effective p value (`spearman_r_eff_p_value`)
* Effective Sample Size (`effective_sample_size`)
* Coefficient of Determination (`r2`)

### Distance-Based

* Root Mean Squared Error (`rmse`)
* Mean Squared Error (`mse`)
* Mean Absolute Error (`mae`)
* Median Absolute Error (`median_absolute_error`)
* Symmetric Mean Absolute Percentage Error (`smape`)
* Mean Absolute Percentage Error (`mape`)

Calling the functions is very straight-forward. All deterministic functions take the form `func(a, b, dim, **kwargs)`. **Notice that the original dataset is reduced by the dimension passed.** I.e., since we passed `time` as the dimension here, we are returned an object with dimensions `(lat, lon)`.

In [4]:
r = xs.pearson_r(obs, fct, "time")
print(r)

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.65498127,  0.2468166 ,  0.86493741,  0.87341323, -0.48728164],
       [ 0.6966213 , -0.45731798, -0.98747468,  0.96668008, -0.42468482],
       [ 0.40086725,  0.75433099, -0.99545503,  0.99634642,  0.04491162],
       [ 0.8985016 , -0.35968423,  0.09314668, -0.5904454 , -0.76273947]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [5]:
p = xs.pearson_r_p_value(obs, fct, "time")
print(p)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.54535301, 0.84123095, 0.33471601, 0.32380295, 0.67597702],
       [0.50937161, 0.69762026, 0.10086578, 0.16480122, 0.72076983],
       [0.73741771, 0.45592281, 0.06071906, 0.05443604, 0.97139875],
       [0.28931331, 0.76576884, 0.9406149 , 0.5979042 , 0.44770775]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also specify multiple axes for deterministic metrics. Here, we apply it over the latitude and longitude dimension (a pattern correlation).

In [6]:
r = xs.pearson_r(obs, fct, ["lat", "lon"])
print(r)

<xarray.DataArray (time: 3)>
array([-0.17520704, -0.2681107 , -0.07783574])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


All deterministic metrics except for `pearson_r_eff_p_value`, `spearman_r_eff_p_value`, and `effective_sample_size` can take the kwarg `weights=...`. `weights` should be a DataArray of the size of the reduced dimension (e.g., if time is being reduced it should be of length 3 in our example).

Weighting is a common practice when working with observations and model simulations of the Earth system. When working with rectilinear grids, one can weight the data by the cosine of the latitude, which is maximum at the equator and minimum at the poles (as in the below example). More complicated model grids tend to be accompanied by a cell area varaible, which could also be passed into this function.

In [7]:
obs2 = xr.DataArray(
       np.random.rand(3, 180, 360),
       coords=[
           xr.cftime_range("2000-01-01", "2000-01-03", freq="D"),
           np.linspace(-89.5, 89.5, 180),
           np.linspace(-179.5, 179.5, 360),
       ],
       dims=["time", "lat", "lon"],
    )
fct2 = obs2.copy()
fct2.values = np.random.rand(3, 180, 360)

In [8]:
# make weights as cosine of the latitude and broadcast
weights = np.cos(np.deg2rad(obs2.lat))
_, weights = xr.broadcast(obs2, weights)

# Remove the time dimension from weights
weights = weights.isel(time=0)

In [9]:
r_weighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=weights)
print(r_weighted)

<xarray.DataArray (time: 3)>
array([ 0.00092601, -0.00028878,  0.00415516])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [10]:
r_unweighted = xs.pearson_r(obs2, fct2, ["lat", "lon"], weights=None)
print(r_unweighted)

<xarray.DataArray (time: 3)>
array([-0.00082695, -0.00087103,  0.00534033])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


You can also pass the optional boolean kwarg `skipna=...`. If `True`, ignore any NaNs (pairwise) in `a` and `b` when computing the result. If `False`, return NaNs anywhere there are pairwise NaNs.

In [11]:
obs_with_nans = obs.where(obs.lat > 1)
fct_with_nans = fct.where(fct.lat > 1)
print(obs_with_nans)

<xarray.DataArray (time: 3, lat: 4, lon: 5)>
array([[[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.12331465, 0.94621187, 0.14512884, 0.868694  , 0.00819766],
        [0.72591448, 0.2831731 , 0.28239725, 0.56488714, 0.27306933]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.23038176, 0.89222197, 0.20256203, 0.30278429, 0.39282783],
        [0.86353827, 0.93015826, 0.4274368 , 0.86071556, 0.95588831]],

       [[       nan,        nan,        nan,        nan,        nan],
        [       nan,        nan,        nan,        nan,        nan],
        [0.50160555, 0.01267184, 0.17515158, 0.81939706, 0.6857781 ],
        [0.79449325, 0.85014403, 0.1019839 , 0.02029124, 0.8486446 ]]])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00
  * lat      (lat) int64 

In [12]:
mae_with_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=True)
print(mae_with_skipna)

<xarray.DataArray (time: 3)>
array([0.37271828, 0.32857743, 0.37334196])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [13]:
mae_without_skipna = xs.mae(obs_with_nans, fct_with_nans, ['lat', 'lon'], skipna=False)
print(mae_without_skipna)

<xarray.DataArray (time: 3)>
array([nan, nan, nan])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Probabilistic Metrics

`xskillscore` offers a suite of probabilistic metrics, mostly ported from `properscoring`.

* Brier Score (`brier_score`)
* Discrimination (`discrimination`)
* Rank Histogram (`rank_histogram`)
* Continuous Ranked Probability Score with the ensemble distribution (`crps_ensemble`)
* Continuous Ranked Probability Score with a Gaussian distribution (`crps_gaussian`)
* Continuous Ranked Probability Score with numerical integration of the normal distribution (`crps_quadrature`)
* Brier scores of an ensemble for exceeding given thresholds (`threshold_brier_score`)


We now create some data with an ensemble member dimension. In this case, we envision an ensemble forecast with multiple members to validate against our theoretical observations.

In [14]:
obs3 = xr.DataArray(
       np.random.rand(4, 5),
       coords=[np.arange(4), np.arange(5)],
       dims=["lat", "lon"]
   )
fct3 = xr.DataArray(
       np.random.rand(3, 4, 5),
       coords=[np.arange(3), np.arange(4), np.arange(5)],
       dims=["member", "lat", "lon"],
   )

Continuous Ranked Probability Score with the ensemble distribution.

In [15]:
crps_ensemble = xs.crps_ensemble(obs3, fct3)
print(crps_ensemble)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.17882095, 0.4965253 , 0.74543606, 0.69028216, 0.45421577],
       [0.11110857, 0.07703227, 0.10797041, 0.38616707, 0.13187368],
       [0.07702046, 0.06046974, 0.50557208, 0.25999662, 0.23665461],
       [0.27481475, 0.15772403, 0.14754376, 0.106625  , 0.65026722]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS with a Gaussian distribution requires two parameters: $\mu$ and $\sigma$ from the forecast distribution. Here, we just use the ensemble mean and ensemble spread.

In [16]:
crps_gaussian = xs.crps_gaussian(obs3, fct3.mean("member"), fct3.std("member"))
print(crps_gaussian)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.12636337, 0.49297531, 0.74384332, 0.68624572, 0.44553368],
       [0.10690331, 0.03904566, 0.06586907, 0.38377942, 0.10232917],
       [0.09498316, 0.06051076, 0.50167733, 0.1666451 , 0.2408734 ],
       [0.27309777, 0.15490502, 0.14653249, 0.07744851, 0.64891212]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


The CRPS quadrature metric requires a callable distribution function. Here we use `norm` from `scipy.stats`.

In [17]:
from scipy.stats import norm
crps_quadrature = xs.crps_quadrature(obs3, norm)
print(crps_quadrature)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.48234353, 0.57190031, 0.53852454, 0.60051146, 0.24776491],
       [0.48769114, 0.43830284, 0.48829526, 0.24000399, 0.26740824],
       [0.51670477, 0.30039312, 0.5459281 , 0.39260831, 0.24675637],
       [0.32469847, 0.23476291, 0.58601938, 0.29590471, 0.53255926]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


We can also use a threshold Brier Score, to score hits over a certain threshold.

In [18]:
threshold_brier_score = xs.threshold_brier_score(obs3, fct3, 0.7)
print(threshold_brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.44444444, 1.        , 1.        , 1.        , 0.11111111],
       [0.44444444, 0.44444444, 0.44444444, 0.11111111, 0.11111111],
       [0.11111111, 0.        , 1.        , 0.11111111, 0.11111111],
       [0.        , 0.        , 0.        , 0.        , 1.        ]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [19]:
brier_score = xs.brier_score(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(brier_score)

<xarray.DataArray (lat: 4, lon: 5)>
array([[0.44444444, 0.44444444, 1.        , 1.        , 1.        ],
       [0.11111111, 0.        , 0.        , 0.44444444, 0.11111111],
       [0.11111111, 0.        , 0.44444444, 0.44444444, 0.44444444],
       [0.        , 0.        , 0.        , 0.11111111, 1.        ]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


In [20]:
rank_histogram = xs.rank_histogram(obs3, fct3)
print(rank_histogram)

<xarray.DataArray 'histogram_rank' (rank: 4)>
array([3, 1, 8, 8])
Coordinates:
  * rank     (rank) float64 1.0 2.0 3.0 4.0


In [21]:
hist_event, hist_no_event = xs.discrimination(obs3 > 0.5, (fct3 > 0.5).mean("member"))
print(hist_event)

<xarray.DataArray (forecast_probability: 5)>
array([0.25      , 0.33333333, 0.        , 0.16666667, 0.25      ])
Coordinates:
  * forecast_probability  (forecast_probability) float64 0.0 0.25 0.5 0.75 1.0


In [22]:
print(hist_no_event)

<xarray.DataArray (forecast_probability: 5)>
array([0.375, 0.25 , 0.   , 0.25 , 0.125])
Coordinates:
  * forecast_probability  (forecast_probability) float64 0.0 0.25 0.5 0.75 1.0


## Contingency-Based

To work with contingency-based scoring, first instantiate a `Contingency` object by passing in your observations, forecast, and observation/forecast bin edges. See https://www.cawcr.gov.au/projects/verification/#Contingency_table for more information.

In [23]:
dichotomous_category_edges = np.array([0, 0.5, 1]) # "dichotomous" mean two-category
dichotomous_contingency = xs.Contingency(obs, fct,
                                        dichotomous_category_edges,
                                        dichotomous_category_edges,
                                        dim=['lat','lon'])
dichotomous_contingency_table = dichotomous_contingency.table

In [24]:
print(dichotomous_contingency_table)

<xarray.DataArray 'histogram_observations_forecasts' (time: 3, observations_category: 2, forecasts_category: 2)>
array([[[4, 6],
        [5, 5]],

       [[3, 7],
        [6, 4]],

       [[2, 6],
        [5, 7]]])
Coordinates:
  * time                          (time) object 2000-01-01 00:00:00 ... 2000-...
    observations_category_bounds  (observations_category) <U10 '(0.0, 0.5]' '...
    forecasts_category_bounds     (forecasts_category) <U10 '(0.0, 0.5]' '(0....
  * observations_category         (observations_category) int64 1 2
  * forecasts_category            (forecasts_category) int64 1 2


Scores based on the constructed contingency table can be called via class methods. The available methods are:

* bias_score
* hit_rate
* false_alarm_ratio
* false_alarm_rate
* success_ratio
* threat_score
* equit_threat_score
* odds_ratio
* odds_ratio_skill_score
* accuracy
* heidke_score
* peirce_score
* gerrity_score

Below, we share a few examples of these in action.

In [25]:
print(dichotomous_contingency.bias_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([1.1       , 1.1       , 1.08333333])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [26]:
print(dichotomous_contingency.hit_rate())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.5       , 0.4       , 0.58333333])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [27]:
print(dichotomous_contingency.false_alarm_rate())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.6 , 0.7 , 0.75])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [28]:
print(dichotomous_contingency.odds_ratio_skill_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.2       , -0.55555556, -0.36363636])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


Now we can leverage multi-category edges to make use of some scores.

In [29]:
multi_category_edges = np.array([0, 0.25, 0.75, 1])
multicategory_contingency = xs.Contingency(obs, fct,
                                           multi_category_edges,
                                           multi_category_edges,
                                           dim=['lat','lon'])

In [30]:
print(multicategory_contingency.accuracy())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([0.25, 0.3 , 0.15])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [31]:
print(multicategory_contingency.heidke_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.12781955, -0.02941176, -0.1971831 ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [32]:
print(multicategory_contingency.peirce_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.12878788, -0.03100775, -0.21875   ])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


In [33]:
print(multicategory_contingency.gerrity_score())

<xarray.DataArray 'histogram_observations_forecasts' (time: 3)>
array([-0.10714286, -0.09451659, -0.16666667])
Coordinates:
  * time     (time) object 2000-01-01 00:00:00 ... 2000-01-03 00:00:00


## Accessors

You can also use `xskillscore` as a method of your `xarray` Dataset.

In [34]:
ds = xr.Dataset()
ds["obs_var"] = obs
ds["fct_var"] = fct

In the case that your Dataset contains both your observation and forecast variable, just pass them as strings into the function.

In [35]:
print(ds.xs.pearson_r("obs_var", "fct_var", "time"))

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.65498127,  0.2468166 ,  0.86493741,  0.87341323, -0.48728164],
       [ 0.6966213 , -0.45731798, -0.98747468,  0.96668008, -0.42468482],
       [ 0.40086725,  0.75433099, -0.99545503,  0.99634642,  0.04491162],
       [ 0.8985016 , -0.35968423,  0.09314668, -0.5904454 , -0.76273947]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4


You can also pass in a separate Dataset that contains your observations or forecast variable.

In [36]:
ds = ds.drop_vars("fct_var")
print(ds.xs.pearson_r("obs_var", fct, "time"))

<xarray.DataArray (lat: 4, lon: 5)>
array([[-0.65498127,  0.2468166 ,  0.86493741,  0.87341323, -0.48728164],
       [ 0.6966213 , -0.45731798, -0.98747468,  0.96668008, -0.42468482],
       [ 0.40086725,  0.75433099, -0.99545503,  0.99634642,  0.04491162],
       [ 0.8985016 , -0.35968423,  0.09314668, -0.5904454 , -0.76273947]])
Coordinates:
  * lat      (lat) int64 0 1 2 3
  * lon      (lon) int64 0 1 2 3 4
